In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import csv
import math
import json
import ast
import cv2
import warnings
import seaborn as sns
import random

np.random.seed(42)
random.seed(42)

# Ignore warnings
warnings.filterwarnings('ignore')

## Dataset 2: one minute - one data point

In [22]:
aggre={
'IndTrig_L' : 'sum',
'IndTrig_R' : 'sum',
'HandTrig_L' : 'sum',
'HandTrig_R' : 'sum',
'Thumb_L_x' : 'sum',
'Thumb_R_x' : 'sum',
'Thumb_L_y' : 'sum',
'Thumb_R_y' : 'sum',
'le_roll' : 'median_change',
'le_pitch' : 'median_change',
'le_yaw' : 'median_change',
'LE_speed' : 'median' ,
'h_roll' : 'median_change',
'h_pitch' : 'median_change',
'h_yaw' : 'median_change',
'Head_Speed' : 'median' , 
'Head_Velocity_Change' : 'median' ,
'Head_AngVel_Change' : 'median',
'c_roll' : 'median_change',
'c_pitch' : 'median_change',
'c_yaw' : 'median_change',
'c_speed' : 'median',
'text_presence' : 'sum',
'change in brightness' : 'sum',
'diff_HE_yaw' : 'median',
'diff_HE_roll' : 'median',
'diff_HE_pitch' : 'median',
'c_acceleration1' : 'median',
'c_acceleration2' : 'median',
'c_acceleration3' : 'median',
'c_velocity1' : 'median',
'c_velocity2' : 'median',
'c_velocity3' : 'median'}

# Ignore warnings
warnings.filterwarnings('ignore')

def expand_cols(df, column_name):
    df[column_name]= df[column_name].apply(lambda x: ast.literal_eval(x))
    data_expanded = pd.DataFrame(df[column_name].tolist(), index=df.index)
    data_expanded.columns = [column_name+'1', column_name+'2', column_name+'3']
    df = df.drop(columns=[column_name]).join(data_expanded)
    return df

def change_diff(df, column_list):
    for column in column_list:
        df[column+'_change']=df[column]-df[column].shift(1)
    return df
    
def aggregate_df(df,aggre):
    df1=expand_cols(df,'c_acceleration')
    df2=expand_cols(df1,'c_velocity')
    change=[]
    for i in aggre.keys():
        if aggre[i]=='median_change':
            change.append(i)
            aggre.pop(i)
            aggre[i+'_change']='median'
    df=change_diff(df2,change)
    aggregated_df = df.agg(aggre)
    return pd.DataFrame(aggregated_df).transpose()


with open('/home/sharedFolder/sessions.json', 'r') as file:
    sessions = json.load(file)

dfs=[]
count=0
session_saving={}

In [23]:
sessions={'Earth_Gym': ['5_2_Earth_Gym',
  '10_2_Earth_Gym']}

In [14]:
def create_frames_1(df_m, aggre,j):
    df=df_m.copy()
    index_list= df[df['MS_rating'].notna()].index.tolist()
    msr=df['MS_rating'].dropna().tolist()
    # print(msr)
    # index_list.insert(0, 0)
    df3=df.iloc[0:1]
    # print(df2)
    # dfs1=df3.copy()
    count1=0
    for i in range(len(index_list) - 1):
        if count1==0:
            start_idx=0
            end_idx=1
            df2=df.iloc[start_idx:end_idx]
            aggre1=aggre.copy()
            df2=aggregate_df(df2.copy(), aggre1)
            # msr=df2['MS_rating'].dropna().max()
            dfs1=df2.copy()
        start_idx = index_list[i]
        end_idx = index_list[i + 1]  
        # df2=df.copy()
        if count==1:
            start_idx+=1
        # print(start_idx, end_idx)
        df2=df.iloc[start_idx:end_idx]
        # print(df2.columns)
        aggre1=aggre.copy()
        df2=aggregate_df(df2.copy(), aggre1)
        # else:
        # print(df2)
        df_c1=[dfs1,df2]
        # return dfs1
        dfs1 = pd.concat(df_c1, ignore_index=True)
        
        # return dfs1
        count1+=1
        # print(count1)
    dfs1['msr']=msr
    dfs1['session']=j
    return dfs1
        

In [25]:
for i in sessions.keys():
    # logger.info(f'***************{i}********************')
    for j in sessions[i]:
        # logger.info(f'*******{j}********')
        path=os.path.join('/data/VR_NET/folders/', i, j, 'data_file_2.csv')
        print(path)
        df1=pd.read_csv(path)
        df=df1.copy()
        # try:
        df=create_frames_1(df, aggre, j)
        # print(df)
        # cleaned_dfs.append(df)
        if count==0:
            dfs=df.copy()
        else:
            df_c=[dfs,df]
            dfs = pd.concat(df_c)
        session_saving[count]=path
        count+=1
            # logger.info('*******successfully saved********')
        # except:
        #     print('failed')
            # logger.info('*******failed********')

/data/VR_NET/folders/Earth_Gym/5_2_Earth_Gym/data_file_2.csv
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
/data/VR_NET/folders/Earth_Gym/10_2_Earth_Gym/data_file_2.csv
[1.0, 1.0, 2.0, 1.0, 3.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0]


# Dataset 3: two_minutes - one data point

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import csv
import math
import json
import ast
import cv2
import warnings
import seaborn as sns
import random
from logger_config import setup_logger

# Get logger
logger = setup_logger()


np.random.seed(42)
random.seed(42)

# Ignore warnings
warnings.filterwarnings('ignore')


aggre={
'IndTrig_L' : 'sum',
'IndTrig_R' : 'sum',
'HandTrig_L' : 'sum',
'HandTrig_R' : 'sum',
'Thumb_L_x' : 'sum',
'Thumb_R_x' : 'sum',
'Thumb_L_y' : 'sum',
'Thumb_R_y' : 'sum',
'le_roll' : 'median_change',
'le_pitch' : 'median_change',
'le_yaw' : 'median_change',
'LE_speed' : 'median' ,
'h_roll' : 'median_change',
'h_pitch' : 'median_change',
'h_yaw' : 'median_change',
'Head_Speed' : 'median' , 
'Head_Velocity_Change' : 'median' ,
'Head_AngVel_Change' : 'median',
'c_roll' : 'median_change',
'c_pitch' : 'median_change',
'c_yaw' : 'median_change',
'c_speed' : 'median',
'text_presence' : 'sum',
'change in brightness' : 'sum',
'diff_HE_yaw' : 'median',
'diff_HE_roll' : 'median',
'diff_HE_pitch' : 'median',
'c_acceleration1' : 'median',
'c_acceleration2' : 'median',
'c_acceleration3' : 'median',
'c_velocity1' : 'median',
'c_velocity2' : 'median',
'c_velocity3' : 'median'}

# Ignore warnings
warnings.filterwarnings('ignore')

def expand_cols(df, column_name):
    df[column_name]= df[column_name].apply(lambda x: ast.literal_eval(x))
    data_expanded = pd.DataFrame(df[column_name].tolist(), index=df.index)
    data_expanded.columns = [column_name+'1', column_name+'2', column_name+'3']
    df = df.drop(columns=[column_name]).join(data_expanded)
    return df

def change_diff(df, column_list):
    for column in column_list:
        df[column+'_change']=df[column]-df[column].shift(1)
    return df
    
def aggregate_df(df,aggre):
    df1=expand_cols(df,'c_acceleration')
    df2=expand_cols(df1,'c_velocity')
    change=[]
    for i in aggre.keys():
        if aggre[i]=='median_change':
            change.append(i)
            aggre.pop(i)
            aggre[i+'_change']='median'
    df=change_diff(df2,change)
    aggregated_df = df.agg(aggre)
    return pd.DataFrame(aggregated_df).transpose()

with open('/home/sharedFolder/sessions.json', 'r') as file:
    sessions = json.load(file)

dfs=[]
count=0

sessions={'Earth_Gym': ['5_2_Earth_Gym',
  '10_2_Earth_Gym']}

In [11]:
def create_frames_1(df_m, aggre,j):
    df=df_m.copy()
    index_list= df[df['MS_rating'].notna()].index.tolist()
    msr=df['MS_rating'].dropna().tolist()
    df3=df.iloc[0:1]
    count1=0
    for i in range(len(index_list) - 2):
        # if count1==0:
        #     start_idx=0
        #     end_idx=1
        #     df2=df.iloc[start_idx:end_idx]
        #     aggre1=aggre.copy()
        #     df2=aggregate_df(df2.copy(), aggre1)
        #     dfs1=df2.copy()
        start_idx = index_list[i]
        end_idx = index_list[i + 2]  
        if count==1:
            start_idx+=1
        df2=df.iloc[start_idx:end_idx]
        aggre1=aggre.copy()
        df2=aggregate_df(df2.copy(), aggre1)
        if count1 ==0:
            dfs1=df2.copy()
        else:
            df_c1=[dfs1,df2]
            dfs1 = pd.concat(df_c1, ignore_index=True)
        count1+=1
        print(start_idx, end_idx)
    print(len(dfs1))
    dfs1['msr']=msr[2:]
    dfs1['msr_1']=msr[1:-1]
    dfs1['msr_2']=msr[:-2]
    dfs1['session']=j
    return dfs1

In [13]:
for i in sessions.keys():
    # logger.info(f'***************{i}********************')
    for j in sessions[i]:
        # logger.info(f'*******{j}********')
        path=os.path.join('/data/VR_NET/folders/', i, j, 'data_file_2.csv')
        print(path)
        df1=pd.read_csv(path)
        df=df1.copy()
        # try:
        df=create_frames_1(df, aggre, j)
        # print(df)
        # cleaned_dfs.append(df)
        if count==0:
            dfs=df.copy()
        else:
            df_c=[dfs,df]
            dfs = pd.concat(df_c)
        # session_saving[count]=path
        count+=1
            # logger.info('*******successfully saved********')
        # except:
        #     print('failed')
            # logger.info('*******failed********')

/data/VR_NET/folders/Earth_Gym/5_2_Earth_Gym/data_file_2.csv
0 588
222 987
588 1337
987 1788
1337 2157
1788 2567
2157 2986
2567 3384
2986 3820
3384 4279
3820 4618
11
/data/VR_NET/folders/Earth_Gym/10_2_Earth_Gym/data_file_2.csv
2 900
430 1306
901 1698
1307 2260
1699 2565
2261 2985
2566 3528
2986 3878
3529 4239
3879 4741
4240 5119
11


In [14]:
dfs

,IndTrig_L,IndTrig_R,HandTrig_L,HandTrig_R,Thumb_L_x,Thumb_R_x,Thumb_L_y,Thumb_R_y,LE_speed,Head_Speed,...,h_roll_change,h_pitch_change,h_yaw_change,c_roll_change,c_pitch_change,c_yaw_change,msr,msr_1,msr_2,session
0,8.000000,8.168498,51.239068,19.478632,17.081848,4.663414,10.019126,6.656926,0.042908,0.049192,...,-0.000797,-0.001815,0.000335,-0.000161,0.000195,0.000000,1.0,1.0,1.0,5_2_Earth_Gym
1,23.629548,151.273748,96.206344,124.476189,42.732780,8.357490,8.872966,1.773924,0.040971,0.040261,...,-0.002170,-0.000581,0.000969,-0.000453,-0.000169,0.000725,1.0,1.0,1.0,5_2_Earth_Gym
2,21.594627,369.824175,66.768496,129.971179,28.309321,4.011695,-3.275341,0.466922,0.048111,0.046984,...,-0.001974,-0.000504,0.001697,-0.000357,0.000218,0.000934,1.0,1.0,1.0,5_2_Earth_Gym
3,99.069597,365.407570,121.239313,62.329412,19.922779,1.139057,2.422207,8.646779,0.081222,0.080344,...,-0.001360,-0.000175,-0.000527,0.002059,0.000284,0.001357,1.0,1.0,1.0,5_2_Earth_Gym
4,107.317948,332.018803,136.711833,76.742109,21.314843,-2.757746,4.034744,1.630032,0.078037,0.076945,...,-0.002277,0.000328,-0.000642,-0.001882,0.000227,0.004574,1.0,1.0,1.0,5_2_Earth_Gym
5,27.312820,432.603174,109.687655,130.765553,4.428476,-1.519324,-0.646609,-0.818831,0.079755,0.079859,...,0.000749,-0.000650,0.000325,-0.007749,0.000120,0.008105,1.0,1.0,1.0,5_2_Earth_Gym
6,39.091331,498.377044,141.991200,296.091804,9.025706,1.005332,-5.483141,0.253941,0.066904,0.068931,...,-0.000538,-0.000077,-0.000019,-0.000793,-0.000209,0.001168,1.0,1.0,1.0,5_2_Earth_Gym
7,67.781685,483.981438,98.408055,355.061285,14.073743,-0.211967,-8.769522,1.705341,0.039280,0.039796,...,-0.000962,0.000049,-0.000481,0.000781,0.000200,0.001077,1.0,1.0,1.0,5_2_Earth_Gym
8,56.564591,494.745297,63.386811,200.623194,16.592093,-0.643063,0.166827,0.574693,0.049953,0.049731,...,-0.001190,0.000752,-0.000425,0.002391,0.000287,0.001111,1.0,1.0,1.0,5_2_Earth_Gym
9,38.465445,423.032722,18.897189,83.182412,28.430035,3.506885,9.264001,-0.580261,0.054485,0.054047,...,-0.003013,0.000947,-0.000161,0.000925,0.000345,0.001596,1.0,1.0,1.0,5_2_Earth_Gym


In [37]:
import argparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import csv
import math
import json
import ast
import cv2
import time
import logging
import warnings
import seaborn as sns
import pickle
import lightgbm as lgb
import xgboost as xgb
import random
import lime
import lime.lime_tabular
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

random.seed(42)
np.random.seed(42)
# from logger_config import setup_logger
global path_for_datafile

# Ignore warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', '{:.10f}'.format)

time_now=str(int(time.time()))

def setup_logger(fname):
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    
    # Configure logging
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    
    # Prevent adding multiple handlers to the logger
    if not logger.hasHandlers():
        fh = logging.FileHandler(fname)
        fh.setLevel(logging.DEBUG)
        ch = logging.StreamHandler()
        ch.setLevel(logging.DEBUG)
        
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)
        
        logger.addHandler(fh)
        logger.addHandler(ch)
    
    return logger


def select_cols(dataset):
    cols=['IndTrig_L', 'IndTrig_R', 'HandTrig_L', 'HandTrig_R', 'Thumb_L_x',
       'Thumb_R_x', 'Thumb_L_y', 'Thumb_R_y', 'Head_Speed','LE_speed',
       'Head_Velocity_Change', 'Head_AngVel_Change', 'c_speed',
       'text_presence', 'change in brightness', 'diff_HE_yaw', 'diff_HE_roll',
       'diff_HE_pitch', 'c_acceleration1', 'c_acceleration2',
       'c_acceleration3', 'c_velocity1', 'c_velocity2', 'c_velocity3',
       'le_roll_change', 'le_pitch_change', 'le_yaw_change', 'h_roll_change',
       'h_pitch_change', 'h_yaw_change', 'c_roll_change', 'c_pitch_change',
       'c_yaw_change','msr_1','time']
    if dataset=='dataset1.csv':
        cols.remove('msr_1')
        cols.remove('time')
    if dataset=='dataset3.csv':
        cols.append('msr_2')
    if dataset=='dataset4.csv':
        cols.append('msr_2')
        cols.append('msr_3')
    return cols

def get_model(modelname,dataset, log):
    model_path=os.path.join('/home/sharedFolder/modelling/', dataset[:-4], log, modelname, 'model.pkl')
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    threshold_path=os.path.join('/home/sharedFolder/modelling/', dataset[:-4], log, modelname, 'threshold.json')
    # Read the threshold value from the JSON file 
    with open(threshold_path, 'r') as json_file: 
        data = json.load(json_file) 
        threshold = data['threshold'] 
    return model, threshold
    
def get_prediction(model,X,threshold): 
    y_prob = model.predict(X)
    y_pred= (y_prob >= threshold).astype(int)
    return y_pred
    
def get_explanation(model, data_filtered, y_pred):
    count=0
    frames=data_filtered['frame']
    X=data_filtered[cols].abs()
    for i in range(len(X)):
        if y_pred[i]==1:
            explainer = lime.lime_tabular.LimeTabularExplainer(X, feature_names=cols, class_names=['msr'], mode='regression')
            conditions = explainer.explain_instance(X[i], model.predict)
            count+=1
            reason, range_values=find_explanation(conditions)
            df = pd.DataFrame({'Reason': reason, 'Range': range_values})
            df['Start']=frame[i]
            df['End']=frame[i+1]
            if count==0:
                df2=df
            else:
                df_c1=[df2,df]
                df2 = pd.concat(df_c1, ignore_index=True)
    return df2

def find_than_signs(text):
    count=0
    for i in text:
        if i=="<" or i=='>':
            count+=1
    return count

def find_explanation(conditions):
    # print(conditions)
    variables=[]
    range=[]
    for condition, coefficient in conditions:
        if coefficient > 0:
            condition=str(condition)
            if 'msr' not in condition and 'time' not in condition:
                # print(condition)
                list1=condition.split(' ')
                count=find_than_signs(condition)
            
                if count==1:
                    variable=list1[0]
                    if '<' in condition:
                        range.append('low')
                    else:
                        range.append('high')
                else:
                    variable=list1[2]
                    if float(list1[0])==0.0 and float(list1[4])==0.0:
                        range.append('low')
                    else:
                        range.append('ranging from '+list1[0]+' to '+ list1[4])
                if list1[0]=='change':
                    variable=list1[0]+' '+list1[1]+' '+list1[2]
                    variables.append(variable)
                elif 'change' in variable:
                    variables.append(variable[:-7])
                elif 'Change' in variable:
                    variables.append(variable[:-7])
                else:
                    variables.append(variable)
    return variables, range

def find_explanation_sum(conditions):
    
    statements=[]
    range1=[]
    dflong=pd.read_csv('/home/sharedFolder/modelling/Codebook.csv')
    for condition, coefficient in conditions:
        list1=condition.split(' ')
        count=find_than_signs(condition)
        if coefficient > 0:
            condition=str(condition)
            if count==1:
                variable=list1[0]
                if list1[0]=='change':
                    variable=list1[0]+' '+list1[1]+' '+list1[2]
                elif ('change' in variable) or ('Change' in variable):
                    variable=variable[:-7]
                long_name=get_long_name([variable], dflong)
                if '<' in condition:
                    statements.append(long_name[0] + ' is low')
                    range1.append('positive')
                else: 
                    statements.append(long_name[0] + ' is high')
                    range1.append('positive')

        elif coefficient < 0:
            condition=str(condition)
            if count==1:
                variable=list1[0]
                if list1[0]=='change':
                    variable=list1[0]+' '+list1[1]+' '+list1[2]
                elif ('change' in variable) or ('Change' in variable):
                    variable=variable[:-7]
                long_name=get_long_name([variable], dflong)
                if '<' in condition:
                    statements.append(long_name[0] + ' is low')
                    range1.append('negative')
                else:
                    statements.append(long_name[0] + ' is high')
                    range1.append('negative')
    return statements, range1

# Define the lookup function
def get_long_name(column_names, df):
    if column_names is None:
        return None
    else:
        # Use .isin() to filter rows where Column is in the list of column_names
        matching_rows = df[df['Column'].isin(column_names)]
        # Reindex the result to match the order of the input column_names
        matching_rows = matching_rows.set_index('Column').reindex(column_names)
        # Get the Long_Name values as a list, replacing NaNs with None
        long_names = matching_rows['Long_Name'].where(pd.notnull(matching_rows['Long_Name']), None).tolist()
        return long_names

def get_min_explanation(y_pred, X, dataful, model, indexes, frame,cols, set1):
    # print(frame)
    count=0
    df2=None
    explainer = lime.lime_tabular.LimeTabularExplainer(dataful.values, feature_names=cols, class_names=['msr'], mode='regression')
    dflong=pd.read_csv('/home/sharedFolder/modelling/Codebook.csv')
    for i in range(len(X)):
        if y_pred[i]==1:
            
            conditions = explainer.explain_instance(X.loc[indexes[i]], model.predict)

            reason, range_values=find_explanation(conditions.as_list())
            long_reason= get_long_name(reason, dflong)
            df = pd.DataFrame({'Reason': reason, 'LongReason': long_reason, 'Range': range_values})
            if count==0:
                df2=df
                df['Start']=frame[i]
                df['End']=frame[i+set1]
            else:
                df['Start']=frame[i]
                df['End']=frame[i+set1]
                df_c1=[df2,df]
                df2 = pd.concat(df_c1, ignore_index=True)
            count+=1
    return df2

def get_summary(session):
    model, threshold=get_model('GradientBoosting','dataset1.csv', 'log_1719881000')
    data2=pd.read_csv('/home/sharedFolder/data/dataset1.csv')
    cols1=select_cols('dataset1.csv')
    data2_1=data2[cols1].abs()
    X_1=data2[data2['session']==session]
    explainer = lime.lime_tabular.LimeTabularExplainer(data2_1.values, feature_names=cols1, class_names=['msr'], mode='regression')
    conditions = explainer.explain_instance(X_1[cols1].abs().values[0], model.predict)
    statement, range1=find_explanation_sum(conditions.as_list())
    y_pred1= model.predict(X_1[cols1].abs())
    if y_pred1[0]<0:
        y_pred1[0]=0
    if y_pred1[0]>1:
        y_pred1[0]=1
    # s1='Overall motion sickness score is '+ str(y_pred1[0]) + ' (the score is in the range of 0-1, where 1 is the highest)'
    s1='Overall motion sickness score is'
    statement.insert(0, s1)
    range1.insert(0, str(y_pred1[0]))
    return statement, range1

def main(modelname, dataset, session, log):
    with open("/home/sharedFolder/sessions.json", 'r') as file:
        games = json.load(file)
    game_name = '_'.join(session.split('_')[2:])
    set1=int(dataset[-5])-1
    
    if (game_name not in games.keys()) or (session not in games[game_name]):
        print("Incorrect gaming session, check whether your session is included in Modelling/sessions.json")
        return None
    path_for_datafile=os.path.join('/data/VR_NET/folders/', game_name, session, 'data_file_2.csv')
    full_data_file=pd.read_csv(path_for_datafile)
    cols=select_cols(dataset)
    data_path=os.path.join('/home/sharedFolder/data', dataset)
    data=pd.read_csv(data_path)
    data_filtered=data[data['session']==session]
    indexes = data_filtered.index
    filtered_df_for_msr = full_data_file[full_data_file['MS_rating'].notnull()]
    # print(filtered_df_for_msr['MS_rating'])
    frame= filtered_df_for_msr['frame'].tolist()
    X=data_filtered[cols].abs()
    model, threshold= get_model(modelname,dataset, log)
    y_pred= get_prediction(model,X,threshold)
    # print(frame,y_pred)
    df=get_min_explanation(y_pred, X, data[cols].abs(), model, indexes, frame,cols, set1)
    for i in range(1, len(df)):
        for j in range(i+1, len(df)):
            if df.loc[i, "Reason"] == df.loc[j, "Reason"] and df.loc[i, "Start"] < df.loc[j, "End"]:
                df.loc[j, "Start"] = df.loc[i, "End"]
    
    root_path1=os.path.join('/home/sharedFolder', game_name, session)
    save_file_at=os.path.join(root_path1, 'Explanation.csv')
    os.makedirs(os.path.dirname(save_file_at), exist_ok=True)
    if df is None:
        output_file = os.path.join(root_path1,'Explanation.txt')
        with open(output_file, 'w') as file:
            file.write('No high motion sickness areas detected')
            print('No high motion sickness areas detected')
    else:
        df.to_csv(save_file_at, index=False)
    
    statement, range1=get_summary(session)
    file_path = os.path.join(root_path1,'summary.csv')
    sum_data = {'summary': statement, 'condition': range1}

    # Create the DataFrame
    df_sum = pd.DataFrame(sum_data)
    df_sum.to_csv(file_path, index=False)
        

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='Script to process a dataset')
#     parser.add_argument('--dataset', metavar='DATASET', type=str,
#                         help='Name of the dataset to process', default='dataset2.csv')
#     parser.add_argument('--session', metavar='SESSION', type=str,
#                         help='Name of the session that you want to explain')
#     parser.add_argument('--modelname', metavar='MODEL', type=str,
#                         help='Name of the model you need the explanation from', default='LightGBM')
#     parser.add_argument('--log', metavar='LOG', type=str,
#                         help='the log file you need the explanation from', default='log_1719881221')
#     args = parser.parse_args()
    
#     main(args.modelname, args.dataset, args.session, args.log)

In [38]:
main('LightGBM','dataset3.csv','46_2_Arena_Clash', 'log_1719915196')

In [43]:
df=pd.read_csv('/home/sharedFolder/Arena_Clash_1/46_2_Arena_Clash/Explanation.csv')

In [44]:
df

,Reason,LongReason,Range,Start,End
0,diff_HE_pitch,Difference of head and eyes in Pitch angle,high,4110,8550
1,diff_HE_roll,Difference of head and eyes in Roll angle,low,4110,8550
2,Head_AngVel,Change in Head Angular Velocity,low,4110,8550
3,c_pitch,Change in Pitch angle of Camera,low,4110,8550
4,Thumb_L_x,Left Thumbstick movement across X axis,ranging from 4.44 to 10.30,4110,8550
5,h_roll,Change in Roll angle of Head,low,4110,8550
6,Thumb_R_y,Right Thumbstick movement across Y axis,high,4110,8550
7,diff_HE_pitch,Difference of head and eyes in Pitch angle,high,8550,11300
8,Head_AngVel,Change in Head Angular Velocity,ranging from 0.09 to 0.13,8550,11300
9,diff_HE_yaw,Difference of head and eyes in Yaw angle,ranging from 0.01 to 0.02,8550,11300


In [42]:
for i in range(1, len(df)):
    for j in range(i+1, len(df)):
        if df.loc[i, "Reason"] == df.loc[j, "Reason"] :
            if df.loc[j, "Start"] < df.loc[i, "End"]:
                df.loc[j, "Start"] = df.loc[i, "End"]

df

,Reason,LongReason,Range,Start,End
0,diff_HE_pitch,Difference of head and eyes in Pitch angle,high,4110,11300
1,change in brightness,Change in brightness,high,4110,11300
2,diff_HE_yaw,Difference of head and eyes in Yaw angle,low,4110,11300
3,le_roll,Change in Roll angle of Eyes,low,4110,11300
4,diff_HE_pitch,Difference of head and eyes in Pitch angle,high,8550,15200
5,c_velocity1,Camera Velocity X axis,low,8550,15200
6,change in brightness,Change in brightness,high,11300,15200
7,h_roll,Change in Roll angle of Head,high,8550,15200
8,diff_HE_pitch,Difference of head and eyes in Pitch angle,high,15200,18370
9,c_yaw,Change in Yaw angle of Camera,low,11300,18370


In [46]:
df=pd.read_csv('/data/VR_NET/folders/Arena_Clash/52_1_Arena_Clash/data_file_4.csv')

In [4]:
!cp /data/VR_NET/folders/Arena_Clash/44_1_Arena_Clash/data_file_4.csv /home/sharedFolder/Arena_Clash/44_1_Arena_Clash/

In [5]:
!cp /data/VR_NET/folders/Arena_Clash/46_2_Arena_Clash/data_file_4.csv /home/sharedFolder/Arena_Clash/46_2_Arena_Clash/

In [6]:
!cp /data/VR_NET/folders/Arena_Clash/50_2_Arena_Clash/data_file_4.csv /home/sharedFolder/Arena_Clash/50_2_Arena_Clash/

In [7]:
!cp /data/VR_NET/folders/Arena_Clash/51_2_Arena_Clash/data_file_4.csv /home/sharedFolder/Arena_Clash/51_2_Arena_Clash/

In [8]:
!cp /data/VR_NET/folders/Arena_Clash/52_1_Arena_Clash/data_file_4.csv /home/sharedFolder/Arena_Clash/52_1_Arena_Clash/

In [56]:
df['MS_rating'].value_counts()

1.0000000000    2
Name: MS_rating, dtype: int64

In [3]:

# Path to the folder you want to zip
folder_to_zip = '/home/sharedFolder/Arena_Clash'

# Path to the output zip file (without the .zip extension)
output_zip_file = '/home/sharedFolder/Arena_Clash.zip'

# Create the zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)

print(f'Folder {folder_to_zip} zipped as {output_zip_file}')

Folder /home/sharedFolder/Arena_Clash zipped as /home/sharedFolder/Arena_Clash.zip


In [59]:
df.columns

Index(['frame', 'timestamp', 'head_dir', 'head_pos', 'head_vel', 'head_angvel',
       'left_eye_dir', 'left_eye_pos', 'left_eye_vel', 'left_eye_angvel',
       ...
       'c_acceleration1', 'c_velocity2', 'c_acceleration2', 'c_velocity3',
       'c_acceleration3', 'text_presence', 'change in brightness',
       'diff_HE_yaw', 'diff_HE_roll', 'diff_HE_pitch'],
      dtype='object', length=113)

In [ ]:
!cp /data/VR_NET/folders/Arena_Clash/51_2_Arena_Clash/data_file_4.csv /home/sharedFolder/Arena_Clash/51_2_Arena_Clash/

In [60]:
train_games=['Earth_Gym', 'Wild_Quest', 'Roommate', 'Waffle_Restaurant', 'UFO_crash_site_venue', 'The_aquarium', 'Kowloon', 'Wake_the_Robot', 'Scifi_Sandbox', 'Citadel', 'Super_Rumble', 'Venues', 'Superhero_Arena', 'Army_Men', 'Bobber_Bay_Fishing', 'Mars_Miners', 'American_Idol', 'Metdonalds', '3D_Play_House', 'Kawaii_Daycare', 'Giant_Paddle_Golf', 'City_Parkour', 'Slash_RPG', 'Pirate_Life']
test_games=['Arena_Clash', 'Zombie', 'VR_Classroom', 'Horizon_Boxing', 'Titanic_Simulation']
val_games = ['Out_Of_Control', 'Geometry_Gunners', 'Barnyard', 'Canyon_Runners', 'Creature_Feature']

In [61]:
len(train_games)

24